# Determining an Average Meteorological Year

This notebook calculates an hourly time series for an average meteorological year (AMY) for a given location, representing the mean weather conditions for a one-year period. This was designed to provide a similar product to a [Typical Meteorological Year](https://nsrdb.nrel.gov/data-sets/tmy), an hourly dataset used for applications in energy and building systems modeling. Because this represents average rather than extreme conditions, an AMY dataset is not suited for designing systems to meet the worst-case conditions occurring at a location.

In the Cal-Adapt: Analytics Engine, there are three options for investigating AMY:  
1. Absolute AMY: represents the historical reference period (1981-2010) or a future reference period based around a 30-year warming level time window. **Note: Absolute AMY data and display is not bias corrected, and should be used for visual data inspection only.**
2. Difference AMY: represents the difference between a designated future 30-year period and the historical baseline (1981-2010)
3. Severe meteorological year: represents the difference between the 90th percentile for a designated future 30-year period and the historical baseline (1981-2010)

Using app.explore.amy() panel, you can easily set how you want to compute the average meteorological year and view a heatmap of the resulting average meteorological year. We also break down the calculation and illustrate how to manually calculate and output a typical month of choice within an average meteorological year.

## Step 0: Setup

Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
%config InlineBackend.figure_format = 'svg' # Make plots look better in the notebook environment 

from climakitae.meteo_yr import (
    compute_amy, # Compute AMY data from gridded data
    compute_severe_yr, # Compute Severe Meteorological Year from gridded data
    lineplot_from_amy_data # Generate lineplot from AMY data 
)
import climakitae as ck
import pandas as pd 
import panel as pn
pn.extension()

Additionally, get set up to make the computing go faster by executing the following cell. It will likely take several minutes to spin up! Learn more about dask and see some common [troubleshooting tips on our FAQ page](https://analytics.cal-adapt.org/docs/faq/).

In [ ]:
from dask.distributed import progress
from climakitae.cluster import Cluster
cluster = Cluster()
cluster.adapt(minimum=0, maximum=16)
client = cluster.get_client()
cluster

To use *climakitae*, load a new application:

In [ ]:
app = ck.Application()

## Step 1: Explore the Average Meteorological Year

The below panel can be used to explore an AMY for a particular variable and geographic area of interest during the historical period, as well as comparisons to an AMY under future conditions at different global warming levels (for more information about warming levels, see the `explore_warming.ipynb` notebook). 

The absolute AMY for a given 30-year period (either the historical period 1981-2010, or a thirty year window centered on where each GCM simulation reaches the specified global warming level) is calculated by identifying for each hour the hourly value that is closest to the mean hourly value across all years to produce a full annual timeseries of hourly data for a variable of interest that best represents the average conditions.

An absolute AMY for the historical period can then be compared to an AMY under different warming levels, and the difference in hourly values of a given variable for an AMY under historical conditions and future conditions can be plotted using the "Difference" button.

The resulting heatmap of AMY values for a full annual timeseries is then plotted, with day of year 1 being January 1st and hour of day given in Pacific Standard Time. A static version of the plot can be saved by using the save button in the lower right of the plot window.

To learn more about the data available on the Analytics Engine, [see our data catalog](https://analytics.cal-adapt.org/data/). 

<span style="color:#FF0000">

**Note:** Once you have selected the options you would like to investigate, please select the "Reload Data" button. It will take 2-4 minutes to update, hang tight!

In [ ]:
app.explore.amy()

## Step 2: Application

Examining a particular month within the Average Meteorological Year can provide hourly information that could inform energy usage analysis. For example, a warm season month can be used to examine cooling demand; while a cold season month can be used for exploring heating demand change under future climate conditions.

To analyze a specific month, we'll retrieve some of the absolute AMY data of interest shown above. Please select in the explore panel above 'Absolute' as your AMY Type, and either 'Historical' or 'Warming Level Future' with a warming level of your choosing. A progress bar is provided when running the cell below, as this step may take a little while. 

#### Retrieve and display the data
First, we'll read in the data from the catalog and simplify the dataset by reducing the simulation and scenario dimensions.

In [ ]:
my_data = app.retrieve() 
my_data = my_data.isel(simulation=0, scenario=0) 
display(my_data)

#### Read the raw hourly data into memory 
First we read in the raw data will be used to compute AMY in the next step. This step may take a while because the hourly data is a quite large.

In [ ]:
my_data = app.load(my_data) 

#### Use the gridded data to compute the Average Meterological Year
We can easily do this using the *climakitae* function `compute_amy`, which is also used under the hood in the panel generated by app.explore.amy(). This step may take a while as well. 

In [ ]:
amy_data = compute_amy(my_data)
display(amy_data)

#### Visualize one month from the Average Meterological Year 
First, we'll subset the dataframe to grab data for the month of January. You can change the month easily by changing the variable `month`; just make sure the value you is the shortened version of the month (i.e. "Jan" instead of "January") so that the code can find the substring in the `amy_data` object. 

In [ ]:
month = 'Jan'
one_month = amy_data[amy_data.index.str.match(month)] # Subset data for just one month 

Next, we'll use the *climakitae* function `lineplot_from_amy_data` to generate a lineplot of our final data. We'll use information about the user inputs to add a descriptive title to the lineplot. 

In [ ]:
amy_selections = app.explore.amy_selections() # Get information about user inputs to AMY panel 

lineplot_from_amy_data(
    one_month, 
    computation_method=amy_selections.computation_method, # Historical or Warming Level Future  
    location_subset=app.selections.cached_area, # Location subset information 
    warmlevel=amy_selections.warmlevel, # Warming level selection 
    variable=app.selections.variable+"("+app.selections.units+")" # Variable and units selected. 
)

#### Use the gridded data to compute the Severe Meterological Year

We can also calculate the 90th percentile of data using the *climakitae* function `compute_severe_yr()`. This step may take a while as well. 

In [ ]:
severe_data = compute_severe_yr(my_data)
display(severe_data)

## Step 3: Export data

To export the Average Meteorologial Year data, use the following code. Likewise, you can also export the Severe Meteorological Year by exporting the 'severe_data' variable instead of 'amy_data'.

In [ ]:
amy_data.to_csv('my_filename' + '.csv')

To export the data that is used to calculate the Average Meteorological Year (not the AMY result), first pick a format from the dropdown menu.

- We recommend NetCDF, which will work with any number of variables and dimensions in your dataset
- CSV works best up to 2-dimensional data (e.g., lon x lat), and will be compressed and exported with a separate metadata file
- GeoTIFF is not possible for Average Meteorological Year as the time series data does not retain a spatial component

In [ ]:
app.export_as()

Next, write in the object you wish to export and your desired filename (in single or double quotation marks).

In [ ]:
app.export_dataset(my_data, 'my_filename')

Lastly, when you are done, close your cluster resources to free them up for the next time you work. 

In [ ]:
client.close()